#Boston 311 v5 - Changing Logistic Regression Labels

IN our previous notebook we tried updating our linear regression model to only use data that was closed within a certain amount of time. This led to a model that had a much higher loss value on the validation set than the training set, which is in contrast to our previous model that had a lower loss value for the validation set than the training set. 

Another method we could use with the linear regression model is assume that all open cases might be closed tomorrow, and assign them a time to close based on that.

For the logistic regression model, we have two important factors:
1. We need to make sure we are not training or validating our model on the cases we want to predict, i.e. the recently opened cases that still have a chance of being closed within the near future. 
2. We can try predictin whether a case will be closed within a certain amount of time, rather than just closed at any point. Basically we can try what we did on the linear regression model in the last notebook and do it for our logistic regression model.

Lastly, the last notebook crashed a few times due to RAM requirements. We can reshape our data cleaning routines to conserve memory by using fewer data copying commands this time.

Below is our open questions and to-dos consolidated from the last notebook. Moving forward we will probably keep this list at the top of each notebook.

##Questions and To-Dos from v4:

2. Add more features
3. clean up the data by removing outliers
6. look at the currently available android app and see what values are available to the user to select, and which categories might be assigned by the 311 agents after receiving a new case.
7. compare a basic model which only uses the department value as a feature to our more complex models as a heuristic for whether additional features actually improve predictions.
8. Moving forward compare our model predictions with the target date assigned by 311 to see which performs better.

Questions to answer:
1. Can we find some basic commonality between open cases?
2. When and how is the target date set? How about the overdue flag?
3. Do cases autoclose after a certain time?


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import tensorflow as tf
import glob
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from datetime import datetime

from IPython.display import display

%matplotlib inline

#first of course we must import the necessary modules

Let's add some code to load the records directly from their storage so we don't have to upload them to google colaboratory each time. So far the URLs appear to be constant.

In [ ]:
url_2023 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/e6013a93-1321-4f2a-bf91-8d8a02f1e62f/download/tmp9g_820k8.csv"
url_2022 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/81a7b022-f8fc-4da5-80e4-b160058ca207/download/tmph4izx_fb.csv"
url_2021 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/f53ebccd-bc61-49f9-83db-625f209c95f5/download/tmppgq9965_.csv"
url_2020 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/script_105774672_20210108153400_combine.csv"
url_2019 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/311_service_requests_2019.csv"
url_2018 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/311_service_requests_2018.csv"
url_2017 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/30022137-709d-465e-baae-ca155b51927d/download/311_service_requests_2017.csv"
url_2016 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/b7ea6b1b-3ca4-4c5b-9713-6dc1db52379a/download/311_service_requests_2016.csv"
url_2015 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/c9509ab4-6f6d-4b97-979a-0cf2a10c922b/download/311_service_requests_2015.csv"
url_2014 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/bdae89c8-d4ce-40e9-a6e1-a5203953a2e0/download/311_service_requests_2014.csv"
url_2013 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/407c5cd0-f764-4a41-adf8-054ff535049e/download/311_service_requests_2013.csv"
url_2012 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/382e10d9-1864-40ba-bef6-4eea3c75463c/download/311_service_requests_2012.csv"
url_2011 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/94b499d9-712a-4d2a-b790-7ceec5c9c4b1/download/311_service_requests_2011.csv"


Let's refactor the code to make fewer copies of the dataframe. Let's also add "scenario" functionality so we can keep code cleaning procedures we tried in previous notebooks. 

In [ ]:
def clean_and_split_for_logistic(myData, scenario) :

  data = myData.copy()
  # Convert the 'open_dt' and 'close_dt' columns to datetime
  data['open_dt'] = pd.to_datetime(data['open_dt'])
  data['closed_dt'] = pd.to_datetime(data['closed_dt'])
  data['survival_time'] = data['closed_dt'] - data['open_dt']
  data['event'] = data['closed_dt'].notnull().astype(int)
  data['ward_number'] = data['ward'].str.extract(r'0*(\d+)')

  

  cols_to_keep = ['case_enquiry_id', 'survival_time', 'event', 'subject', 'reason', 'department', 'source', 'ward_number']
  cols_to_drop = [
 'open_dt',
 'target_dt',
 'closed_dt',
 'ontime',
 'case_status',
 'closure_reason',
 'case_title',
 'type',
 'queue',
 'submittedphoto',
 'closedphoto',
 'location',
 'fire_district',
 'pwd_district',
 'city_council_district',
 'police_district',
 'neighborhood',
 'neighborhood_services_district',
 'ward',
 'precinct',
 'location_street_name',
 'location_zipcode',
 'latitude',
 'longitude']

  #scenarios
  #scenario 1: drop any open cases from the last month, and switch the event value for any cases that took longer than a month to close
  if (scenario == 1) :
    # Convert the date string to a pandas Timestamp object
    cutoff_date = pd.Timestamp('2023-04-09')

    # Filter the DataFrame to include only rows where event is 1 or open_dt is before the cutoff date
    data = data[(data['event'] == 1) | (data['open_dt'] < cutoff_date)]

    #switch the event value for any cases that took longer than a month to close
    delta = pd.Timedelta(seconds=2678400)
    data.loc[(data['event'] == 1) & (data['survival_time'] > delta), 'event'] = 0


  data = data.drop(cols_to_drop, axis=1)

  data = pd.get_dummies(data, columns=['subject', 'reason', 'department', 'source', 'ward_number'])



  #fix this line to also drop the case_enquiry_id
  X = data.drop(['case_enquiry_id','event', 'survival_time'], axis=1)
  y = data['event']

  return X, y

def clean_and_split_for_linear(myData, scenario) :

  data = myData.copy()
  # Convert the 'open_dt' and 'close_dt' columns to datetime
  data['open_dt'] = pd.to_datetime(data['open_dt'])
  data['closed_dt'] = pd.to_datetime(data['closed_dt'])
  data['survival_time'] = data['closed_dt'] - data['open_dt']
  data['event'] = data['closed_dt'].notnull().astype(int)
  data['ward_number'] = data['ward'].str.extract(r'0*(\d+)')

  cols_to_keep = ['case_enquiry_id', 'survival_time', 'event', 'subject', 'reason', 'department', 'source', 'ward_number']
  cols_to_drop = [
 'open_dt',
 'target_dt',
 'closed_dt',
 'ontime',
 'case_status',
 'closure_reason',
 'case_title',
 'type',
 'queue',
 'submittedphoto',
 'closedphoto',
 'location',
 'fire_district',
 'pwd_district',
 'city_council_district',
 'police_district',
 'neighborhood',
 'neighborhood_services_district',
 'ward',
 'precinct',
 'location_street_name',
 'location_zipcode',
 'latitude',
 'longitude']

  data = data.drop(cols_to_drop, axis=1)

  data = pd.get_dummies(data, columns=['subject', 'reason', 'department', 'source', 'ward_number'])
  data_survival_mask = data["survival_time"].notnull()
  clean_data = data[data_survival_mask].copy()
  clean_data['survival_time_hours'] = clean_data['survival_time'].apply(lambda x: x.total_seconds()/3600)

  #add scenarios

  #scenario 1: remove records if the case took more than a month to close or time to close is negative
  if (scenario == 1) :
    clean_data = clean_data[(clean_data['survival_time_hours'] >= 0) & (clean_data['survival_time_hours'] <= 744)]
  
  #scenario 2: remove records just if the time to close is negative 
  if (scenario == 2) :
    clean_data = clean_data[(clean_data['survival_time_hours'] >= 0)]

  #fix this line to also drop the case_enquiry_id
  X = clean_data.drop(['case_enquiry_id','survival_time_hours', 'survival_time', 'event'], axis=1) 
  y = clean_data['survival_time_hours']
  
  return X, y

Here is the link to all the data sets:

https://data.boston.gov/dataset/311-service-requests

In [ ]:

# Get a list of all CSV files in the directory
all_files = [url_2023, url_2022, url_2021, url_2020, url_2019, url_2018, url_2017, url_2016, url_2015, url_2014, url_2013, url_2012, url_2011]

# Create an empty list to store the dataframes
dfs = []

# Loop through the files and load them into dataframes
for file in all_files:
  df = pd.read_csv(file)
  dfs.append(df)

<ipython-input-3-c7e6c2df8ad4>:9: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
<ipython-input-3-c7e6c2df8ad4>:9: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


In [ ]:
#check that the files all have the same number of columns, and the same names
for i in range(len(dfs)):
  if dfs[i].shape[1] != dfs[0].shape[1]:
    print('Error: File', i, 'does not have the same number of columns as File 0')
  else:
    print('File', i, 'has same number of columns as File 0')
  if not dfs[i].columns.equals(dfs[0].columns):
    print('Error: File', i, 'does not have the same column names and order as File 0')
  else:
    print('File', i, 'has the same column name and order as File 0')

File 0 has same number of columns as File 0
File 0 has the same column name and order as File 0
File 1 has same number of columns as File 0
File 1 has the same column name and order as File 0
File 2 has same number of columns as File 0
File 2 has the same column name and order as File 0
File 3 has same number of columns as File 0
File 3 has the same column name and order as File 0
File 4 has same number of columns as File 0
File 4 has the same column name and order as File 0
File 5 has same number of columns as File 0
File 5 has the same column name and order as File 0
File 6 has same number of columns as File 0
File 6 has the same column name and order as File 0
File 7 has same number of columns as File 0
File 7 has the same column name and order as File 0
File 8 has same number of columns as File 0
File 8 has the same column name and order as File 0
File 9 has same number of columns as File 0
File 9 has the same column name and order as File 0
File 10 has same number of columns as Fi

In [ ]:
# Concatenate the dataframes into a single dataframe
df_all = pd.concat(dfs, ignore_index=True)

In [ ]:
#save ram by deleting the dfs variable
del dfs

In [ ]:
df_all.columns.tolist()

['case_enquiry_id',
 'open_dt',
 'target_dt',
 'closed_dt',
 'ontime',
 'case_status',
 'closure_reason',
 'case_title',
 'subject',
 'reason',
 'type',
 'queue',
 'department',
 'submittedphoto',
 'closedphoto',
 'location',
 'fire_district',
 'pwd_district',
 'city_council_district',
 'police_district',
 'neighborhood',
 'neighborhood_services_district',
 'ward',
 'precinct',
 'location_street_name',
 'location_zipcode',
 'latitude',
 'longitude',
 'source']

In [ ]:
logistic_X, logistic_y = clean_and_split_for_logistic(df_all, 1)

In [ ]:
linear_X, linear_y = clean_and_split_for_linear(df_all, 2)

In [36]:
linear_X.describe()



,subject_Animal Control,subject_Boston Police Department,subject_Boston Water & Sewer Commission,subject_CRM Application,subject_Consumer Affairs & Licensing,subject_Disability Department,subject_Inspectional Services,subject_Mayor's 24 Hour Hotline,subject_Neighborhood Services,subject_Parks & Recreation Department,...,ward_number_20,ward_number_21,ward_number_22,ward_number_3,ward_number_4,ward_number_5,ward_number_6,ward_number_7,ward_number_8,ward_number_9
count,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,...,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06
mean,5.866812e-03,1.621427e-04,3.971850e-03,4.300866e-07,3.440693e-06,9.083429e-04,7.542085e-02,3.842265e-02,4.859979e-05,5.733227e-02,...,5.820921e-02,4.030600e-02,4.248224e-02,8.972897e-02,4.273599e-02,8.104767e-02,5.123837e-02,4.257384e-02,3.411490e-02,3.039981e-02
std,7.637012e-02,1.273250e-02,6.289735e-02,6.558099e-04,1.854908e-03,3.012504e-02,2.640693e-01,1.922144e-01,6.971187e-03,2.324765e-01,...,2.341387e-01,1.966760e-01,2.016867e-01,2.857931e-01,2.022613e-01,2.729084e-01,2.204836e-01,2.018944e-01,1.815243e-01,1.716848e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [37]:
linear_y.describe()

count    2.325113e+06
mean     3.958667e+02
std      1.789195e+03
min      0.000000e+00
25%      1.416944e+00
50%      1.710167e+01
75%      1.176658e+02
max      6.989429e+04
Name: survival_time_hours, dtype: float64

In [38]:
#Train a logistic regression model

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(logistic_X, logistic_y, test_size=0.2, random_state=42)

# Build model
model_logistic = keras.Sequential([
    keras.layers.Dense(units=1, input_shape=(X_train.shape[1],), activation='sigmoid')
])

# Compile model
model_logistic.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model_logistic.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate model
test_loss, test_acc = model_logistic.evaluate(X_test, y_test)

print('Test accuracy:', test_acc)

Epoch 1/10
64032/64032 [==============================] - 111s 2ms/step - loss: 0.3003 - accuracy: 0.8633
Epoch 2/10
64032/64032 [==============================] - 123s 2ms/step - loss: 0.2958 - accuracy: 0.8654
Epoch 3/10
64032/64032 [==============================] - 109s 2ms/step - loss: 0.2958 - accuracy: 0.8657
Epoch 4/10
64032/64032 [==============================] - 107s 2ms/step - loss: 0.2958 - accuracy: 0.8657
Epoch 5/10
64032/64032 [==============================] - 112s 2ms/step - loss: 0.2958 - accuracy: 0.8655
Epoch 6/10
64032/64032 [==============================] - 107s 2ms/step - loss: 0.2958 - accuracy: 0.8657
Epoch 7/10
64032/64032 [==============================] - 113s 2ms/step - loss: 0.2958 - accuracy: 0.8655
Epoch 8/10
64032/64032 [==============================] - 109s 2ms/step - loss: 0.2958 - accuracy: 0.8657
Epoch 9/10
64032/64032 [==============================] - 122s 2ms/step - loss: 0.2958 - accuracy: 0.8655
Epoch 10/10
16008/16008 [=====================

Let's delete the df_all variable to save RAM before we train our linear regression model

In [39]:
del df_all

In [40]:
#Train a linear regression model

start_time = datetime.now()
print("Starting Training at {}".format(start_time))

scaler = StandardScaler()
X_scaled = scaler.fit_transform(linear_X) # scale the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, linear_y, test_size=0.2, random_state=42)

# split the data again to create a validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# define the model architecture
model_linear = keras.Sequential([
    keras.layers.Dense(units=1, input_dim=X_train.shape[1])
])

# compile the model
model_linear.compile(optimizer='adam', loss='mean_squared_error')

# train the model
model_linear.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1, validation_data=(X_val, y_val))

end_time = datetime.now()
total_time = (end_time - start_time)
print("Ending Training at {}".format(end_time))
print("Training took {}".format(total_time))

Starting Training at 2023-05-09 16:59:26.027579
Epoch 1/50
46503/46503 [==============================] - 108s 2ms/step - loss: 3270872.0000 - val_loss: 3272695.5000
Epoch 2/50
46503/46503 [==============================] - 84s 2ms/step - loss: 3174310.5000 - val_loss: 3191508.0000
Epoch 3/50
46503/46503 [==============================] - 84s 2ms/step - loss: 3110366.7500 - val_loss: 3141374.2500
Epoch 4/50
46503/46503 [==============================] - 93s 2ms/step - loss: 3103389.2500 - val_loss: 3111857.0000
Epoch 5/50
46503/46503 [==============================] - 92s 2ms/step - loss: 3087338.5000 - val_loss: 3088312.0000
Epoch 6/50
46503/46503 [==============================] - 91s 2ms/step - loss: 3086873.2500 - val_loss: 3079226.2500
Epoch 7/50
46503/46503 [==============================] - 84s 2ms/step - loss: 3100280.0000 - val_loss: 3069662.7500
Epoch 8/50
46503/46503 [==============================] - 83s 2ms/step - loss: 3102633.7500 - val_loss: 3059529.5000
Epoch 9/50
4650

In [41]:
model_linear.save("model_linear.h5")

In [42]:
model_logistic.save("model_logistic.h5")

We save these models so that we can start building a website and API to make these models available to the public